## NOTE - Use Python 3.6

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import pymongo
import dill
import os

from BrattEssay import load_bratt_essays
from load_data import load_process_essays
from window_based_tagger_config import get_config
from FindFiles import find_files
from DirUtils import dir_exists
from results_procesor import ResultsProcessor, __MICRO_F1__
from Settings import Settings
from statsmodels.stats.contingency_tables import mcnemar

In [2]:
FOLDER = "/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Predictions/"

## Find Closest Perceptron Results for CB

In [14]:
with open(FOLDER + "TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FINAL_RUN_VD_YS_.dill", "rb+") as f:
    cb_ysbytag = dill.load(f)
    
with open(FOLDER + "TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_FINAL_RUN_VD_YS_.dill", "rb+") as f:
    sc_ysbytag = dill.load(f)

In [15]:
def filter_by_str(s, files, exclude=False):   
    return  [f for f in files if (s in f) != exclude]

In [16]:
# FILTER = "STACKED"
FILTER = "FINAL_RUN"

In [23]:
all_files = os.listdir(FOLDER)
# print(len(all_files))
perc_files = all_files
perc_files = filter_by_str("_CB_", all_files)
perc_files = filter_by_str("_CR_", perc_files)
perc_files = filter_by_str("TEST_", perc_files)
perc_files = filter_by_str("_VD_", perc_files)
perc_files = filter_by_str("_PREDS", perc_files)
perc_files = filter_by_str("_" + FILTER + "_", perc_files)
# perc_files = filter_by_str("2019", perc_files)
print(len(perc_files))
perc_files

7


['TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FINAL_RUN_VD_PREDS__.dill',
 'TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FINAL_RUN_VD_PREDS__2019-10-11_17-25.dill',
 'TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FINAL_RUN_VD_PREDS__2019-10-11_17-24.dill',
 'TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FINAL_RUN_VD_PREDS__2019-10-11_17-23.dill',
 'TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FINAL_RUN_VD_PREDS__2019-10-11_17-22.dill',
 'TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FINAL_RUN_VD_PREDS__2019-10-11_17-6.dill',
 'TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FINAL_RUN_VD_PREDS__2019-10-11_17-26.dill']

In [24]:
import pandas as pd

ysbytag = cb_ysbytag
f2preds = dict()
rows = []
for f in perc_files[::-1]:
    with open(FOLDER + f, "rb+") as fin:
        preds = dill.load(fin)
    metrics = ResultsProcessor.compute_mean_metrics(ysbytag, preds)[__MICRO_F1__]
    metrics["fname"] = f[-16:]
    rows.append(metrics)
df = pd.DataFrame(rows).sort_values("f1_score")
df # 0.837

,accuracy,data_points,f1_score,fname,num_codes,precision,recall
2,0.997836,164948.0,0.719119,10-11_17-22.dill,599.0,0.680060,0.762938
4,0.997848,164948.0,0.722005,10-11_17-24.dill,599.0,0.679941,0.769616
5,0.997860,164948.0,0.722702,10-11_17-25.dill,599.0,0.682493,0.767947
1,0.997908,164948.0,0.726408,-10-11_17-6.dill,599.0,0.691843,0.764608
3,0.997921,164948.0,0.730558,10-11_17-23.dill,599.0,0.689911,0.776294
6,0.997921,164948.0,0.730558,_VD_PREDS__.dill,599.0,0.689911,0.776294
0,0.997939,164948.0,0.730586,10-11_17-26.dill,599.0,0.695324,0.769616


## SC 

In [21]:
all_files = os.listdir(FOLDER)
perc_files = all_files
perc_files = filter_by_str("_SC_", perc_files)
perc_files = filter_by_str("_VD_", perc_files)
perc_files = filter_by_str("TEST_", perc_files)
perc_files = filter_by_str("_PREDS_", perc_files)
perc_files = filter_by_str("_" + FILTER + "_", perc_files)
# perc_files = filter_by_str("_2019", perc_files)
perc_files

['TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_FINAL_RUN_VD_PREDS_.dill',
 'TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_FINAL_RUN_VD_PREDS__2019-10-11_17-27.dill',
 'TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_FINAL_RUN_VD_PREDS__2019-10-11_17-26.dill',
 'TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_FINAL_RUN_VD_PREDS__2019-10-11_17-23.dill',
 'TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_FINAL_RUN_VD_PREDS__2019-10-11_17-7.dill',
 'TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_FINAL_RUN_VD_PREDS__2019-10-11_17-22.dill',
 'TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_FINAL_RUN_VD_PREDS__2019-10-11_17-25.dill',
 'TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_FINAL_RUN_VD_PREDS__2019-10-11_17-29.dill',
 'TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_FINAL_RUN_VD_PREDS__2019-10-11_17-28.dill',
 'TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_FINAL_RUN_VD_PREDS__2019-10-11_17-24.dill']

In [22]:
import pandas as pd

ysbytag = sc_ysbytag
f2preds = dict()
rows = []
for f in perc_files:
    with open(FOLDER + f, "rb+") as fin:
        preds = dill.load(fin)
    metrics = ResultsProcessor.compute_mean_metrics(ysbytag, preds)[__MICRO_F1__]
    metrics["fname"] = f[-16:]
    rows.append(metrics)
df = pd.DataFrame(rows).sort_values("f1_score")
df # 0.837

,accuracy,data_points,f1_score,fname,num_codes,precision,recall
5,0.995514,102753.0,0.788823,10-11_17-22.dill,1118.0,0.808451,0.770125
0,0.995552,102753.0,0.790847,N_VD_PREDS_.dill,1118.0,0.809747,0.772809
7,0.995552,102753.0,0.790847,10-11_17-29.dill,1118.0,0.809747,0.772809
9,0.995582,102753.0,0.791552,10-11_17-24.dill,1118.0,0.813208,0.771020
3,0.995552,102753.0,0.791610,10-11_17-23.dill,1118.0,0.807442,0.776386
1,0.995591,102753.0,0.791724,10-11_17-27.dill,1118.0,0.814570,0.770125
6,0.995552,102753.0,0.791800,10-11_17-25.dill,1118.0,0.806871,0.777281
4,0.995562,102753.0,0.791971,-10-11_17-7.dill,1118.0,0.808194,0.776386
8,0.995582,102753.0,0.792315,10-11_17-28.dill,1118.0,0.810861,0.774597
2,0.995572,102753.0,0.792332,10-11_17-26.dill,1118.0,0.808947,0.776386
